## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv").set_index("City_ID")
vacation_df.drop(columns="City_ID.1",inplace=True)
vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Rating
City_ID,,,,,,,,,,,
0,Ixtapa,20.7000,-105.2000,75.22,89,100,5.01,MX,2022-01-19 23:50:07,Hotel Plaza Providencia,4.4
1,Karaton,-6.3424,106.1045,79.95,93,97,5.17,ID,2022-01-19 23:50:07,Parkir Mobil Wisata CAS Water Park,4.3
2,Ushuaia,-54.8000,-68.3000,53.26,58,0,17.27,AR,2022-01-19 23:45:13,Albatross Hotel,4.3
3,Busselton,-33.6500,115.3333,78.04,52,2,9.98,AU,2022-01-19 23:50:08,Observatory Guest House,4.5
4,Pacific Grove,36.6177,-121.9166,65.43,75,0,1.01,US,2022-01-19 23:50:08,Lovers Point Inn,3.7


In [3]:
country_counts_df = vacation_df.groupby('Country').count().sort_values('City', ascending = False)[['City']]
country_counts_df.head()

,City
Country,
RU,47
US,47
AU,38
BR,31
ID,25


In [4]:
def check_hotel_count(country):
    return country_counts_df.loc[country,:].values[0]
check_hotel_count('GA')

4

In [5]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Rating</dt><dd>{Hotel Rating}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [6]:
# 4a. Add a marker layer for each city to the map. 
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.
import dataframe_image as dfi

def get_random_itinerary(number, country):
    if (country == ""):
        #print(country_counts_df.loc[(country_counts_df['City_ID'] >= int(number))])
        country = country_counts_df.loc[(country_counts_df['City'] >= int(number))].sample(1).index[0]
        print(f"We chose the country {country} at random for you!")
    
    if (country != ""):
        hotel_count = check_hotel_count(country)
        if (hotel_count >= number):
            temp = vacation_df.loc[(vacation_df['Country']==country)]
            #print(temp)
            itinerary_df = temp.sample(number)
            dfi.export(itinerary_df, "{country}_random_itinerary.png")
            return itinerary_df
            
        else:
            user_input = input(f"Sorry, we only found {hotel_count} Hotels in {country}, (y) to continue with this country, otherwise to exit")
            if (user_input == "y"):
                get_random_itinerary(hotel_count, country)
                return
            else:
                return      

In [8]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.
vacation = get_random_itinerary(4,"")

vacation_start = vacation_df.iloc[0,:]
vacation_end = vacation_df.iloc[0,:]
vacation_stop1 = vacation_df.iloc[1,:]
vacation_stop2 = vacation_df.iloc[2,:]
vacation_stop3 = vacation_df.iloc[3,:]

vacation

We chose the country PF at random for you!


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Rating
City_ID,,,,,,,,,,,
24,Rikitea,-23.1203,-134.9692,76.57,77,100,7.14,PF,2022-01-19 23:50:12,People ThankYou,4.2
443,Fare,-16.7000,-151.0167,80.56,77,100,22.30,PF,2022-01-20 00:00:53,Maitai Lapita Village Huahine,4.5
167,Faanui,-16.4833,-151.7500,80.62,78,100,23.26,PF,2022-01-19 23:53:48,Bora-Bora Pearl Beach Resort & Spa,4.7
149,Moerai,-22.4333,-151.3333,78.98,65,100,10.38,PF,2022-01-19 23:53:44,Le Manotel,3.6


In [9]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
def get_lat_lng(df):
    return df[['Lat','Lng']].to_numpy()
get_lat_lng(get_random_itinerary(4,""))

We chose the country BS at random for you!


array([[ 26.5412, -77.0636],
       [ 26.6208, -78.2833],
       [ 24.0333, -74.5167],
       [ 23.1   , -74.9833]])

In [10]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
import gmaps.datasets

def create_directions_layer(coords, style):
    fig = gmaps.figure()
    
    start = coords[0]
    
    if style != "one-way":
        style = "loop"
        end = start
        my_directions_layer = gmaps.directions_layer(start,end, waypoints=coords[1:], travel_mode='DRIVING', show_markers=False)
    else:
        end = coords[-1]
        my_directions_layer = gmaps.directions_layer(start, end, waypoints=coords[1:-1], travel_mode='DRIVING', show_markers=False)
    
    return my_directions_layer

In [11]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
def format_df_markers(df):
    info_box_template = """
    <dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>Rating</dt><dd>{Hotel Rating}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
    </dl>
    """

    # 10a Get the data from each row and add it to the formatting template and store the data in a list.
    formatted_data = [info_box_template.format(**row) for index, row in df.iterrows()]
    return formatted_data

    # 10b. Get the latitude and longitude from each row and store in a new DataFrame.
def draw_itinerary(df,style):
    locations = df[["Lat", "Lng"]]
    
    hotel_info = format_df_markers(df)
    
    # 11a. Add a marker layer for each city to the map.
    max_temp = df["Max Temp"]
    fig = gmaps.figure()
    directions_layer = create_directions_layer(get_lat_lng(df),style)
    marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
    heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
                 max_intensity=300, point_radius=4)
    fig.add_layer(directions_layer)
    fig.add_layer(heat_layer)
    fig.add_layer(marker_layer)    
    
# 11b. Display the figure
    return fig

In [12]:
get_random_itinerary(4,"")

We chose the country PE at random for you!


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Rating
City_ID,,,,,,,,,,,
689,Cajaruro,-5.7364,-78.4256,77.85,76,92,3.65,PE,2022-01-20 00:06:49,Hotel Kuelap,3.9
318,Paita,-5.0892,-81.1144,75.76,78,98,14.20,PE,2022-01-19 23:57:23,Hotel Náutico de Paita,3.8
407,Huicungo,-7.2833,-76.8000,75.60,85,98,2.55,PE,2022-01-19 23:59:44,Gran Pajaten Trek,4.0
168,Pisco,-13.7000,-76.2167,75.25,73,0,21.85,PE,2022-01-19 23:53:48,Hotel San Isidro Oficial,4.0


In [13]:
draw_itinerary(get_random_itinerary(4,""),"one-way")

We chose the country CV at random for you!


Figure(layout=FigureLayout(height='420px'))

In [21]:
draw_itinerary(get_random_itinerary(6,"US"),"loop")

Figure(layout=FigureLayout(height='420px'))

In [15]:
import googlemaps
import simplejson
import json
import numpy as np

gdistance = googlemaps.Client(key = g_key)

def get_distance_matrix(coords):
    durations = []
    result = gdistance.distance_matrix(coords,coords, mode='driving')
    
    #print(json.dumps(result, indent = 4))
    
    df = pd.DataFrame(index=result['origin_addresses'],
                      columns=result['destination_addresses'])
    
    for i in range(len(coords)):
        for j in range(len(coords)):
            try:
                #This is the duration of the travel time in seconds. Divide by 3600 for H:M:S
                df.iloc[i,j] = result["rows"][i]["elements"][j]["duration"]["value"]
            except (KeyError):
                df.iloc[i,j] = np.nan    
    return df
    
    #result = gdistance.distance_matrix(origin, destination, mode='driving')["rows"][0]["elements"][0]["duration"]["value"]  
    #duration = duration/3600
    #durations.append(duration)
    
    #return durations

In [16]:
from sys import maxsize
from itertools import permutations
 
# implementation of traveling Salesman Problem
def travellingSalesmanProblem(graph, start, style):
    if start == "":
        start = 0
        
    # store minimum weight Hamiltonian Cycle
    min_path = maxsize
    path_length = 0
    best_walk = ""
    short_path = ()
    
    for perm in permutations(range(len(graph))):
        if perm[0] != start:
            continue
        log=""
        path_length = 0
        log +=f"{perm}\n"
        for i in perm:
            if i != perm[-1]:
                path_length += graph[i][perm[perm.index(i)+1]]
                log += f"Walking from node {i} to node {perm[perm.index(i)+1]} at a cost of {graph[i][perm[perm.index(i)+1]]}, path length is now: {path_length}\n"
            elif (style == "loop"):
                path_length += graph[i][perm[0]]
                log += f"Walking from node {i} to node {perm[0]} at a cost of {graph[i][perm[0]]}, path length is now: {path_length}\n"
        # print(log)
        if path_length < min_path:
            min_path = path_length
            short_path = perm
            best_walk = log
    print(f"{best_walk}")
    return [short_path,min_path]

# matrix representation of graph
graph = [[0, 10, 15, 20], [10, 0, 35, 25],
        [15, 35, 0, 30], [20, 25, 30, 0]]
travellingSalesmanProblem(graph,"","")

(0, 1, 3, 2)
Walking from node 0 to node 1 at a cost of 10, path length is now: 10
Walking from node 1 to node 3 at a cost of 25, path length is now: 35
Walking from node 3 to node 2 at a cost of 30, path length is now: 65



[(0, 1, 3, 2), 65]

In [17]:
#Here are other datasets easily available from gmaps
gmaps.datasets.list_datasets()

dict_keys(['taxi_rides', 'earthquakes', 'acled_africa', 'acled_africa_by_year', 'london_congestion_zone', 'nuclear_plants', 'starbucks_kfc_uk', 'gini'])

In [18]:
def get_random_sorted_itinerary(number, country, style):
    itinerary = get_random_itinerary(number,country)
    #print(itinerary)
    new_order = list(travellingSalesmanProblem(get_distance_matrix(get_lat_lng(itinerary)).values,"",style)[0])
    #print(new_order)
    sorted_itinerary = itinerary.copy()
    for i in range(len(itinerary)):
        sorted_itinerary.iloc[i,:] = itinerary.iloc[new_order[i],:]
    #print(sorted_itinerary)
    #return sorted_itinerary
    return draw_itinerary(itinerary,style);

In [22]:
get_random_sorted_itinerary(5,"","loop")

We chose the country MX at random for you!
(0, 1, 4, 2, 3)
Walking from node 0 to node 1 at a cost of 48365, path length is now: 48365
Walking from node 1 to node 4 at a cost of 81824, path length is now: 130189
Walking from node 4 to node 2 at a cost of 87294, path length is now: 217483
Walking from node 2 to node 3 at a cost of 61778, path length is now: 279261
Walking from node 3 to node 0 at a cost of 28436, path length is now: 307697



Figure(layout=FigureLayout(height='420px'))